# core

> This is a core library for the ERA5 dataset pipeline. It defines
a few helpful functions such as an API tester to test your API key and connection.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import cdsapi
import hydra

from omegaconf import DictConfig, OmegaConf
from pyprojroot import here

In [ ]:
#| exporti
def _expand_path(
        path: str   # Path on user's machine
        )->   str:  # Expanded path
    "Expand the path on the user's machine for cross compatibility"

    # Expand ~ to the user's home directory
    path = os.path.expanduser(path)
    # Expand environment variables
    path = os.path.expandvars(path)
    # Convert to absolute path
    path = os.path.abspath(path)
    return path

In [ ]:
#| export
def describe():
    print("This package fetches ERA5 data.")

In [ ]:
#| export
#@hydra.main(version_base=None, config_path=here() / "conf", config_name="config")
def testAPI(
    output_path:str=None,
    dataset:str="reanalysis-era5-pressure-levels",
    remove:bool=False,
    cfg: DictConfig = None
    )-> bool:    
    
    #print(OmegaConf.to_yaml(cfg))

    try:
        client = cdsapi.Client()

        # check the path
        if output_path is None:
            output_path = here() / "data"
        else:
            output_path = _expand_path(output_path)

        if not os.path.exists(output_path):
            os.makedirs(output_path)

        # build request
        request = {
            'product_type': ['reanalysis'],
            'variable': ['geopotential'],
            'year': ['2024'],
            'month': ['03'],
            'day': ['01'],
            'time': ['13:00'],
            'pressure_level': ['1000'],
            'data_format': 'grib',
        }

        target = output_path / 'download.grib'
        
        print("Testing API connection by downloading a dummy dataset to {}...".format(output_path))

        client.retrieve(dataset, request, target)

        if remove:
            os.remove(target)
        
        print("API connection test successful.")
        return True

    except Exception as e:
        print("API connection test failed.")
        print("Did you set up your API key with CDS? If not, please visit https://cds.climate.copernicus.eu/api-how-to")
        print("Error: {}".format(e))
        return False

In [ ]:
#export
if __name__ == "__main__":
    describe()
    testAPI()
else:
    pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()